### 关于有效温度

> 一般定义的有效温度

对每一个温度分布计算其有效温度
$$
T_{eff} = \frac{\int T(X) dX/X [exp(-X/\Lambda_{\pi}) - exp(-X/\Lambda_{N})]}{\int dX/X [exp(-X/\Lambda_{\pi}) - exp(-X/\Lambda_{N})]}
$$

其中 $\Lambda_{\pi} = 160gm/cm^2$ , $\Lambda_{N} = 120gm/cm^2$

> 对于有效温度的优化目标

我们挺希望新定义的有效温度能够最大地反映$\mu$子通量的变化，也就是得最大化相关系数

$$
\alpha(d) = \frac{T(d)}{I_{\mu}} \frac{d I_{\mu}}{T(d)} = \frac{dln{I_{\mu}}}{dlnT(d)}
$$

也就是要让$dlnT(d)$最小，有效温度函数的对数对着时间的偏导数最小

$$
T(d) =\frac{\int_0^{\infty} T(d,h)f(h) dh}{\int_0^{\infty} f(h) dh}
$$

即：
$$
\frac{\partial lnT(d)}{\partial d} = \frac{\partial}{\partial d} ln(\int_0^{\infty} T(d,h)f(h) dh)
$$
$$
\frac{\frac{\partial}{\partial d} \int_0^{\infty} T(d,h)f(h) dh}{\int_0^{\infty} T(d,h)f(h) dh} = 0
$$
在这里，交换分子微分与积分的顺序：
$$
\int_0^{\infty} \frac{\partial}{\partial d} T(d,h)f(h) dh = 0
$$

通过这个方程得到权的分布应该就是最好的$f(h)$, 为了求解这个积分方程，我们对权函数做更多的规定：

- $f(h)$最好是单调的

- $f(h)$最好是非负的

- $f(h)$必须是绝对可积的

- $f(h)$最好是标度无关的

同时为了衡量后面得到的不同权函数的优劣，我们用一个因子衡量：
$$
M = \int_0^{T} dt \int_0^{\infty} \frac{\partial}{\partial t} T(t,h)f(h) dh
$$

$$
m = \frac{\int_0^{T} dt \int_0^{\infty} \frac{\partial}{\partial t} T(t,h)f(h) dh}
{\int_0^{T} dt \int_0^{\infty} \frac{\partial}{\partial t} T(t,h) dh \int f(h)dh} 
$$
其中T为一年的周期，显然M越小，相关程度越高

In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.interpolate import interp1d
from MCEq.geometry import density_profiles
import gc

msis_atmospheres = np.array([
            ["SouthPole", -90,0],              # 南极点
            ["Karlsruhe",49.0097,8.4044],     # 卡尔斯鲁厄，德国
            ["Geneva",46.2044,6.1432],        # 日内瓦，瑞士
            ["Tokyo",35.6762,139.6503],       # 东京，日本
            ["SanGrasso",42.42,13.57],        # Gran Sasso 实验室,意大利
            ["TelAviv",32.0853,34.7818],      # 特拉维夫，以色列
            ["KSC",28.5721,-80.6480],         # 肯尼迪航天发射中心，美国
            ["SoudanMine",47.8167,-92.2333],  # 苏丹矿，美国
            ["Tsukuba",36.0833,140.0833],     # 筑波，日本
            ["LynnLake",56.85,-101.05],       # 林恩湖，加拿大
            ["PeaceRiver",56.2333,-117.2833], # 和平河镇，加拿大
            ["FtSumner",34.4717,-104.2453]    # 萨姆纳堡，美国
        ])


def effective_temperature(weight_func,loc = 'SouthPole'):
    tmp_field = np.zeros((1000,365))
    effective_temperatures = np.zeros(365)
    heights = np.linspace(0,50,1000)*1e2*1e3
    weights = weight_func(heights)

    for doy in range(365):
        tmp_model = density_profiles.MSIS00Atmosphere(
            location = loc,
            doy = doy)
        tmp_distribution = [tmp_model.get_temperature(h) for h in heights]
        tmp_field[:,doy] = tmp_distribution

        intting_func1 = interp1d(heights,tmp_distribution*weights,kind='cubic')
        quadded1,_ = quad(intting_func1,heights[0],heights[-1])
        intting_func2 = interp1d(heights,weights,kind='cubic')
        quadded2,_ = quad(intting_func2,heights[0],heights[-1])
        effective_temperatures[doy] = quadded1/quadded2

        print(f'\rGenerating Doy {doy+1} temperature field: {(doy+1)/365 :.1%}',
              end='', flush=True)
    print('\n',flush=False)

    M_factors = np.zeros(365-1)
    M_divides = np.zeros(365-1)
    partialt_temp = np.diff(tmp_field,n=1,axis=1)
    for ddoy in range(365-1):
        intting_func1 = interp1d(heights,weights*partialt_temp[:,ddoy],kind='cubic')
        M_factor,_ = quad(intting_func1,heights[0],heights[-1])
        M_factors[ddoy] = M_factor

        intting_func2 = interp1d(heights,partialt_temp[:,ddoy],kind='cubic')
        M_divides[ddoy],_ = quad(intting_func2,heights[0],heights[-1])

        print(f'\rGenerating Doy {ddoy+1} diff temperature field: {(ddoy+1)/364 :.1%}',
              end='', flush=True)
        
    print("\n", flush=False)

    intting_func1 = interp1d(np.arange(364)+0.5,M_factors,kind='cubic')
    quadded1,_ = quad(intting_func1,0.5,363.5)
    intting_func2 = interp1d(np.arange(364)+0.5,M_divides,kind='cubic')
    quadded2,_ = quad(intting_func2,0.5,363.5)
    intting_func3 = interp1d(heights,weights,kind='cubic')
    quadded3,_ = quad(intting_func3,heights[0],heights[-1])

    M = quadded1/(quadded2*quadded3)

    del tmp_field
    gc.collect()
    
    return effective_temperatures,M

def weight1(h):
    return np.exp(-h/(2*h[-1]))
eT,m = effective_temperature(weight1,loc = "KSC")

Generating Doy 365 temperature field: 100.0%

Generating Doy 364 diff temperature field: 100.0%



In [2]:
m

8.409068496692203e-08

> 最简单的情况，$f(h)$ = Const $\neq$ 0, $T = \bar{T}$

这里的$T_{eff}$就是对h的平均温度